In [ ]:
!pip install google-generativeai transformers

In [ ]:
!pip install pydantic

In [ ]:
import google.generativeai as genai
import json
import re
from typing import Dict, List, Any, Tuple
import textwrap
import random

In [ ]:
API_KEY = ""
genai.configure(api_key=API_KEY)

In [ ]:
generation_config = {
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [ ]:
from pydantic import BaseModel

class Profile(BaseModel):
    interests: List[str]
    skills: List[str]
    personality_traits: List[str]
    values: List[str]
    education_level: str
    experience_level: str
    dislikes: List[str]


In [ ]:
class CareerRecommendation(BaseModel):
    career_name: str
    fit_explanation: str
    required_skills_education: str
    potential_growth: str

class CareerRecommendationsResponse(BaseModel):
    recommendations: List[CareerRecommendation]
    additional_advice: str

In [ ]:
class DynamicCareerGuidanceAgent:
    def __init__(self):
        self.conversation_history = []
        self.career_demand=[]
        self.personality_type=""
        self.user_profile = {
            "interests": [],
            "skills": [],
            "personality_traits": [],
            "values": [],
            "education_level": "",
            "experience_level": "",
            "dislikes": []
        }

        # Initial system prompt
        self.system_prompt = """
        You are a career guidance expert helping students and professionals find suitable career paths.
        Your goal is to ask insightful questions to understand the person's interests, skills, personality,
        values, education level, and experience. Based on their responses, you should generate follow-up
        questions to dive deeper into aspects that are most relevant for career guidance.

        After gathering enough information, you should analyze their profile and suggest 3-5 suitable
        career paths with explanations for why each career might be a good fit.

        Important: Always be encouraging, professional, and focused on helping the person discover
        careers that align with their unique combination of traits and aspirations.
        """

        # Add system prompt to conversation history
        self.conversation_history.append({"role": "system", "parts": [self.system_prompt]})

    def generate_question(self) -> str:
        """Generate a dynamic question based on conversation history using Gemini"""
        try:
            # Create prompt for question generation
            prompt_parts = [
                "Based on the conversation so far, generate the next question to better understand the person ",
                "for career guidance purposes. Focus on aspects that haven't been covered yet or need more detail.",
                "The question should be open-ended and encourage thoughtful responses.",
                "Conversation history:",
                str(self.conversation_history),
                "Generate just the question without any additional text:"
            ]

            prompt = "\n".join(prompt_parts)
            response = model.generate_content(prompt)
            return response.text.strip()

        except Exception as e:
            # Fallback questions if API call fails
            fallback_questions = [
                "What are you most passionate about?",
                "What skills do you enjoy using the most?",
                "Describe your ideal work environment.",
                "What kind of problems do you enjoy solving?",
                "What values are most important to you in a career?",
                "What subjects did you enjoy most in school?",
                "What are your long-term career aspirations?"
            ]
            return random.choice(fallback_questions)

    def extract_profile_info(self,question:str, response: str):
        """Extract key information from user response to build profile"""
        try:
            prompt = f"""
            Extract key information from this user response for career guidance purposes:

            Question:
            {question}
            Response:
            {response}

            Extract and categorize the following information if mentioned:
            1. Interests/hobbies/passions
            2. Skills/abilities
            3. Personality traits
            4. Values/career preferences
            5. Education level
            6. Experience level

            Return the information in JSON format with these categories.
            If a category isn't mentioned, set it to an empty list or empty string.
            """

            result = model.generate_content(
                prompt,
                generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                response_schema=Profile)
            )

            # Try to parse JSON from response
            try:
                extracted_data = json.loads(result.text)
                print(extracted_data)
                # Update user profile with extracted data
                for key, value in extracted_data.items():
                    if key in self.user_profile:
                        if isinstance(value, list):
                            self.user_profile[key].extend(value)
                        elif isinstance(value, str) and value:
                            if key in ["education_level", "experience_level"]:
                                self.user_profile[key] = value
                            else:
                                self.user_profile[key].append(value)

            except json.JSONDecodeError:
                # If JSON parsing fails, use keyword matching as fallback
                print("JSON parsing failed. Using keyword matching.")
                self.fallback_extraction(response)

        except Exception as e:
          print("Error extracting profile information:", e)
          self.fallback_extraction(response)

    def fallback_extraction(self, response: str):
        """Fallback method for extracting profile information using keyword matching"""
        text = response.lower()

        # Simple keyword matching for interests
        interest_keywords = {
            "technology": ["tech", "computer", "programming", "coding", "software", "ai", "machine learning"],
            "arts": ["art", "design", "creative", "painting", "drawing", "music", "writing"],
            "science": ["science", "research", "experiment", "biology", "chemistry", "physics"],
            "business": ["business", "finance", "marketing", "management", "entrepreneur"],
            "healthcare": ["health", "medical", "doctor", "nurse", "patient", "medicine"],
            "education": ["teach", "education", "learn", "student", "school"],
            "sports": ["sport", "athlete", "fitness", "exercise", "game"],
            "social": ["help", "community", "social", "people", "care"]
        }

        for interest, keywords in interest_keywords.items():
            if any(keyword in text for keyword in keywords):
                if interest not in self.user_profile["interests"]:
                    self.user_profile["interests"].append(interest)

        # Extract other information with simple patterns
        if "student" in text or "study" in text:
            self.user_profile["education_level"] = "Student"
        elif "graduate" in text or "degree" in text:
            self.user_profile["education_level"] = "Graduate"
        elif "experience" in text or "work" in text or "job" in text:
            self.user_profile["experience_level"] = "Experienced"

    def generate_recommendations(self) -> str:
        """Generate career recommendations based on the user profile"""
        try:
            prompt = f"""
            Based on the following user profile, suggest 3-5 suitable career paths with explanations:

            User Profile:
            {json.dumps(self.user_profile, indent=2)}

            Return strictly in this JSON schema
            """

            response = model.generate_content(
                prompt,
                generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                response_schema=CareerRecommendationsResponse)
            )
            extracted_data = json.loads(response.text)
            recommendations = CareerRecommendationsResponse(**extracted_data)
            return recommendations


        except Exception as e:
            return "I apologize, but I'm having trouble generating recommendations at the moment. Please try again later."

    def conduct_session(self, num_questions: int = 8):
        """Conduct a career guidance session with dynamic questioning"""
        print("=" * 60)
        print("WELCOME TO THE DYNAMIC CAREER GUIDANCE AGENT")
        print("=" * 60)
        print("I'll ask you a series of questions to understand your interests,")
        print("skills, and personality to suggest suitable career paths.")
        print("Please answer as honestly and thoroughly as you can.\n")

        # Ask initial questions to get started
        initial_questions = [
            "To get started, could you tell me a bit about your current educational background or work experience?",
            "What are some of your main interests or hobbies?",
            "What skills do you feel are your strongest?",
            "How would you describe your personality in a few words?"
        ]

        for question in initial_questions:
            print(f"\n{question}")
            response = input("Your answer: ")

            # Add to conversation history
            self.conversation_history.append({"role":"system","parts":[question]})
            self.conversation_history.append({"role": "user", "parts": [response]})

            # Extract profile information
            self.extract_profile_info(question,response)

        # Ask dynamic questions based on previous responses
        for i in range(num_questions - len(initial_questions)):
            question = self.generate_question()
            print(f"\n{question}")
            response = input("Your answer: ")

            # Add to conversation history
            self.conversation_history.append({"role":"system","parts":[question]})
            self.conversation_history.append({"role": "user", "parts": [response]})

            # Extract profile information
            self.extract_profile_info(question, response)

        # Generate and display recommendations
        print("\n" + "=" * 60)
        print("THANK YOU FOR YOUR RESPONSES!")
        print("I'm analyzing your profile to suggest career paths...")
        print("=" * 60)

        recommendations = self.generate_recommendations()

        print("\nHERE ARE YOUR PERSONALIZED CAREER RECOMMENDATIONS:")
        print("=" * 60)
        for idx, rec in enumerate(recommendations.recommendations, 1):
          print(f"{idx}. {rec.career_name}")
          print(f"   - Why it's a good fit: {rec.fit_explanation}")
          print(f"   - Required skills: {rec.required_skills_education}")
          print(f"   - Growth: {rec.potential_growth}\n")
        print(f"Additional advice: {recommendations.additional_advice}")

        # Offer to save results
        save = input("\nWould you like to save your results to a file? (yes/no): ")
        if save.lower() in ['yes', 'y']:
            filename = "career_guidance_report.txt"
            with open(filename, 'w') as f:
                f.write("DYNAMIC CAREER GUIDANCE REPORT\n")
                f.write("=" * 50 + "\n\n")
                f.write("USER PROFILE:\n")
                f.write(json.dumps(self.user_profile, indent=2))
                f.write("\n\nRECOMMENDATIONS:\n")
                f.write(recommendations)
            print(f"Report saved to {filename}")

In [ ]:
agent=DynamicCareerGuidanceAgent()
agent.conduct_session(5)

WELCOME TO THE DYNAMIC CAREER GUIDANCE AGENT
I'll ask you a series of questions to understand your interests,
skills, and personality to suggest suitable career paths.
Please answer as honestly and thoroughly as you can.


To get started, could you tell me a bit about your current educational background or work experience?
{'dislikes': [], 'education_level': 'B.Tech in Computer Science', 'experience_level': 'Student', 'interests': [], 'personality_traits': [], 'skills': [], 'values': []}

What are some of your main interests or hobbies?
{'dislikes': [], 'education_level': '', 'experience_level': '', 'interests': ['watching tech videos'], 'personality_traits': [], 'skills': [], 'values': []}

What skills do you feel are your strongest?
{'dislikes': [], 'education_level': '', 'experience_level': '', 'interests': [], 'personality_traits': [], 'skills': ['logical thinking'], 'values': []}

How would you describe your personality in a few words?
{'dislikes': [], 'education_level': '', 'expe

In [ ]:
agent.conversation_history

In [ ]:
agent.generate_recommendations()